<a href="https://colab.research.google.com/github/IshaqKHATTAK/Natural-Language-Processing/blob/main/English_to_urdu_machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [128]:
!unzip '/content/drive/MyDrive/urd-eng.zip'

Archive:  /content/drive/MyDrive/urd-eng.zip
replace urd.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: urd.txt                 
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: _about.txt              


#Data preprocessing

In [144]:
import keras
from keras.layers import Embedding
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot, Tokenizer
from keras.layers import LSTM
from keras.layers import Dense




In [131]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "urd.txt"

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [132]:
input_texts[:5]

['Hi.', 'Help!', 'Thanks.', 'We won.', 'Beat it.']

In [133]:
print(target_character)

['\t', '\n', '(', ')', '/', '،', '؟', 'آ', 'ؤ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'و', 'ي', 'ً', 'ُ', 'ِ', 'ّ', '٘', 'ٹ', 'پ', 'چ', 'ڈ', 'ڑ', 'ژ', 'ک', 'گ', 'ں', 'ھ', 'ہ', 'ی', 'ے', '۔', '۰', '۱', '۲', '۳', '۵', '۷', '۸', '۹']


In [134]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [135]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)


Number of samples: 1140
Number of unique input tokens: 69
Number of unique output tokens: 68
Max sequence length for inputs: 94
Max sequence length for outputs: 88


In [136]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])

target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [137]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '(': 5,
 ')': 6,
 '-': 7,
 '.': 8,
 '/': 9,
 '،': 10,
 '؟': 11,
 'آ': 12,
 'ؤ': 13,
 'ئ': 14,
 'ا': 15,
 'ب': 16,
 'ت': 17,
 'ث': 18,
 'ج': 19,
 'ح': 20,
 'خ': 21,
 'د': 22,
 'ذ': 23,
 'ر': 24,
 'ز': 25,
 'س': 26,
 'ش': 27,
 'ص': 28,
 'ض': 29,
 'ط': 30,
 'ظ': 31,
 'ع': 32,
 'غ': 33,
 'ف': 34,
 'ق': 35,
 'ل': 36,
 'م': 37,
 'ن': 38,
 'و': 39,
 'ي': 40,
 'ً': 41,
 'ُ': 42,
 'ِ': 43,
 'ّ': 44,
 '٘': 45,
 'ٹ': 46,
 'پ': 47,
 'چ': 48,
 'ڈ': 49,
 'ڑ': 50,
 'ژ': 51,
 'ک': 52,
 'گ': 53,
 'ں': 54,
 'ھ': 55,
 'ہ': 56,
 'ی': 57,
 'ے': 58,
 '۔': 59,
 '۰': 60,
 '۱': 61,
 '۲': 62,
 '۳': 63,
 '۵': 64,
 '۷': 65,
 '۸': 66,
 '۹': 67}

In [138]:
import numpy as np
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [139]:
encoder_input_data.shape

(1140, 94, 69)

In [119]:
for k, j in enumerate(target_texts[0]):
  print(k)
  print(j)

0
	
1
س
2
ل
3
ا
4
م
5
۔
6




In [140]:
decoder_input_data.shape

(1140, 88, 68)

In [141]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

#Model building

In [145]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

#training model

In [146]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
15/15 [==============================] - 14s 132ms/step - loss: 1.8454 - accuracy: 0.6879 - val_loss: 2.7810 - val_accuracy: 0.5814
Epoch 2/100
15/15 [==============================] - 0s 23ms/step - loss: 1.3367 - accuracy: 0.7393 - val_loss: 2.1745 - val_accuracy: 0.5814
Epoch 3/100
15/15 [==============================] - 0s 21ms/step - loss: 1.2678 - accuracy: 0.7409 - val_loss: 2.0923 - val_accuracy: 0.5857
Epoch 4/100
15/15 [==============================] - 0s 23ms/step - loss: 1.1939 - accuracy: 0.7411 - val_loss: 2.4995 - val_accuracy: 0.5840
Epoch 5/100
15/15 [==============================] - 0s 26ms/step - loss: 1.1292 - accuracy: 0.7437 - val_loss: 1.8357 - val_accuracy: 0.5857
Epoch 6/100
15/15 [==============================] - 0s 29ms/step - loss: 1.1221 - accuracy: 0.7442 - val_loss: 1.7694 - val_accuracy: 0.5857
Epoch 7/100
15/15 [==============================] - 0s 22ms/step - loss: 1.0750 - accuracy: 0.7442 - val_loss: 1.6940 - val_accuracy: 0.5851
Epoc

#Run inference (sampling)

In [147]:
#1. encode input and retrieve initial decoder state
#2. run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token.
#3. Repeat with the current target token and current states

# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [148]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 20ms/step
-
Input sentence: Hi.
Decoded sentence: میں اپنے اس کو کو کر نے کی کر کی ہے۔

1/1 [==============================] - 0s 23ms/step
-
Input sentence: Help!
Decoded sentence: میں اپنے اس کو کو کر نے کی کر کی ہے۔

1/1 [==============================] - 0s 42ms/step
-
Input sentence: Thanks.
Decoded sentence: میں اپنے اس کو کو کر نے کی کر کی ہے۔

1/1 [==============================] - 0s 20ms/step
-
Input sentence: We won.
Decoded sentence: میں اپنے اس کو کو کر نے کی کر کی ہے۔

1/1 [==============================] - 0s 21ms/step
-
Input sentence: Beat it.
Decoded sentence: میں اپنے اس کو کو کر نے کی کر کی ہے۔

1/1 [==============================] - 0s 27ms/step
-
Input sentence: Beat it.
Decoded sentence: میں اپنے اس کو کو کر نے کی کر کی ہے۔

1/1 [==============================] - 0s 32ms/step
-
Input sentence: We lost.
Decoded sentence: میں اپنے اس کو کو کر نے کی کر کی ہے۔

1/1 [==============================] - 0s 19ms/step
-
Input sente